In [4]:
import numpy as np
import xarray as xr
from src.score import *

In [2]:
datadir = '/data/stephan/WeatherBench/5.625deg/'

In [18]:
z500_valid = load_test_data(f'{datadir}geopotential_500', 'z').load()
t850_valid = load_test_data(f'{datadir}temperature_850', 't').load()

In [16]:
preds = xr.open_dataset(
    '/home/rasp/data/myWeatherBench/predictions/45-resnet_deeper2_no_ss.nc'
).load()

In [8]:
compute_weighted_rmse(preds.z, z500_valid).load()

<xarray.DataArray 'z_rmse' ()>
array(348.37885482)
Coordinates:
    level    int32 500

In [92]:
def compute_weighted_acc(da_fc, da_true):
    clim = da_true.mean('time')
    t = np.intersect1d(da_fc.time, da_true.time)
    fa = da_fc.sel(time=t) - clim
    a = da_true.sel(time=t) - clim
    
    weights_lat = np.cos(np.deg2rad(da_fc.lat))
    weights_lat /= weights_lat.mean()
    w = weights_lat
    
    fa_prime = fa - fa.mean()
    a_prime = a - a.mean()
    
    acc = (
        np.sum(w * fa_prime * a_prime) /
        np.sqrt(
            np.sum(w * fa_prime**2) * np.sum(w * a_prime**2)
        )
    )
    return acc

In [95]:
compute_weighted_acc(preds.z, z500_valid)

<xarray.DataArray ()>
array(0.94556074)
Coordinates:
    level    int32 500

In [96]:
compute_weighted_acc(preds.t, t850_valid)

<xarray.DataArray ()>
array(0.93661655)
Coordinates:
    level    int32 850